In [1]:
# Import gurobi and numpy
from gurobipy import *
import numpy as np
from numpy import genfromtxt
import csv

## Get index of 4 tickers
tick4 = ["MSFT","GS","PG","SCHP"];

# Get variable names
with open('Prices.csv') as csvFile:
    reader = csv.reader(csvFile)
    tickers = next(reader) ## stores the tickers of all 390 stocks

tickind =[];
for t in tick4:
    tickind.append(tickers.index(t)) ## retrieve index that corresponds to each ticker

# Load data
prices = genfromtxt('Prices.csv', delimiter=',',skip_header = 1)

# get dimensions of data
d = prices.shape[0]
n = prices.shape[1]

# calculate monthly returns of each stock
returns = np.zeros((d-1,n))
for stock in range(n):
    for month in range(d-1):
        returns[month,stock] = prices[month+1,stock]/prices[month,stock]-1
        
# Store average return (parameter r_i in portfolio optimization model)       
avg_return = np.zeros(n)
avg_return = np.mean(returns,axis=0)

# Store covariance matrix (parameter C_ij in portfolio optimization model)
C = np.zeros((n,n))
C = np.cov(np.transpose(returns))

In [3]:
avg_return.shape

(390,)

In [4]:
returns.shape

(60, 390)

In [5]:
C.shape

(390, 390)

In [7]:
prices.shape

(61, 390)

In [9]:
prices

array([[138. ,  38.2,  52. , ...,  46. ,  52.7,  80.1],
       [128. ,  36.7,  49.2, ...,  48.1,  54. ,  81.3],
       [132. ,  39.3,  50.1, ...,  50.6,  54.5,  81.5],
       ...,
       [202. ,  64. ,  95.8, ...,  63.3,  54.5,  79.2],
       [213. ,  66.6,  93.8, ...,  65.7,  54.3,  78.9],
       [214. ,  71.4,  94.8, ...,  64.3,  53.3,  77.7]])

In [10]:
tickind

[315, 216, 372, 388]

In [12]:
avg_return_4 = avg_return[tickind]

In [14]:
returns_4 = returns[:,tickind]

In [18]:
C_4 = C[tickind]

In [21]:
C_4_fin = C_4[:,tickind]

In [24]:
C_4_fin.shape

(4, 4)

In [25]:
C_4_fin

array([[ 2.73159720e-03,  1.17211178e-03,  6.24877966e-04,
        -1.38713291e-04],
       [ 1.17211178e-03,  3.72663251e-03, -1.27036282e-05,
        -2.38168523e-04],
       [ 6.24877966e-04, -1.27036282e-05,  1.35693681e-03,
         1.06900756e-04],
       [-1.38713291e-04, -2.38168523e-04,  1.06900756e-04,
         1.18934539e-04]])

In [34]:
mod = Model()

# y = mod.addVars(6, 5, 5, vtype=GRB.BINARY)
w = mod.addVars(4)
rho = 0.005

mod.addConstr(sum([w[i]*avg_return_4[i] for i in range(4)]) >= rho)
mod.addConstr(sum(w[i] for i in range(4)) == 1)
mod.addConstrs(w[i] >= 0 for i in range(4))

# for i in range(6):
#     mod.addConstr(sum([y[i,k,j]*num_hours_or_daily_kj[k][j] for j in range(5) for k in range(5)])
#                   >= new_allocation_hr_t[i]-s[i])
#     mod.addConstr(s[i]>=0)
    
# for k in range(5):
#     for j in range(5):
#         mod.addConstr(sum([y[i,k,j] for i in range(6)]) == 1)
    
mod.setObjective(sum(w[i]*w[j]*C_4_fin[i][j] for j in range(4) for i in range(4)), GRB.MINIMIZE)

mod.update()

mod.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 4 columns and 12 nonzeros
Model fingerprint: 0x70c79b8a
Model has 10 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-04, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-05, 7e-03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 4 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 8 nonzeros
Presolved model has 10 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 3
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.93770406e+03 -2.93770406e+03  4.

In [36]:
w

{0: <gurobi.Var C0 (value 0.23711755759355474)>,
 1: <gurobi.Var C1 (value 0.02585959640455055)>,
 2: <gurobi.Var C2 (value 4.619508687870176e-08)>,
 3: <gurobi.Var C3 (value 0.7370227998068113)>}

In [37]:
mod = Model()

# y = mod.addVars(6, 5, 5, vtype=GRB.BINARY)
w = mod.addVars(390)
rho = 0.005

mod.addConstr(sum([w[i]*avg_return[i] for i in range(390)]) >= rho)
mod.addConstr(sum(w[i] for i in range(390)) == 1)
mod.addConstrs(w[i] >= 0 for i in range(390))

# for i in range(6):
#     mod.addConstr(sum([y[i,k,j]*num_hours_or_daily_kj[k][j] for j in range(5) for k in range(5)])
#                   >= new_allocation_hr_t[i]-s[i])
#     mod.addConstr(s[i]>=0)
    
# for k in range(5):
#     for j in range(5):
#         mod.addConstr(sum([y[i,k,j] for i in range(6)]) == 1)
    
mod.setObjective(sum(w[i]*w[j]*C[i][j] for j in range(390) for i in range(390)), GRB.MINIMIZE)

mod.update()

mod.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 392 rows, 390 columns and 1170 nonzeros
Model fingerprint: 0xd3a53ccf
Model has 76245 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-06, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 2 rows, 390 columns, 780 nonzeros
Presolved model has 76245 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 59
 AA' NZ     : 1.830e+03
 Factor NZ  : 1.891e+03
 Factor Ops : 7.753e+04 (less than 1 second per iteration)
 Threads    : 8

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   1.10520633e-17 

In [38]:
w

{0: <gurobi.Var C0 (value 9.258984417990638e-08)>,
 1: <gurobi.Var C1 (value 1.0667414274817902e-07)>,
 2: <gurobi.Var C2 (value 0.011364021182681032)>,
 3: <gurobi.Var C3 (value 0.006654149998946985)>,
 4: <gurobi.Var C4 (value 1.4758191545448923e-07)>,
 5: <gurobi.Var C5 (value 0.013674340326578075)>,
 6: <gurobi.Var C6 (value 1.7120097502661654e-07)>,
 7: <gurobi.Var C7 (value 3.605693402402651e-08)>,
 8: <gurobi.Var C8 (value 9.763240755215913e-08)>,
 9: <gurobi.Var C9 (value 1.5234982542983273e-07)>,
 10: <gurobi.Var C10 (value 5.991947188236462e-08)>,
 11: <gurobi.Var C11 (value 1.5053792570755887e-07)>,
 12: <gurobi.Var C12 (value 8.075352567885663e-08)>,
 13: <gurobi.Var C13 (value 1.1971426749238925e-07)>,
 14: <gurobi.Var C14 (value 1.0461019413951737e-07)>,
 15: <gurobi.Var C15 (value 8.252467471166364e-08)>,
 16: <gurobi.Var C16 (value 1.0382184624497949e-07)>,
 17: <gurobi.Var C17 (value 1.0253829376518925e-07)>,
 18: <gurobi.Var C18 (value 1.2862787302741345e-07)>,
 19: <

In [40]:
mod = Model()

l = mod.addVars(390, vtype=GRB.BINARY)
w = mod.addVars(390)
z = mod.addVars(390)

rho = 0.005

mod.addConstr(sum([w[i]*l[i]*avg_return[i] for i in range(390)]) >= rho)
mod.addConstr(sum(w[i]*l[i] for i in range(390)) == 1)
mod.addConstrs(w[i] >= 0 for i in range(390))
mod.addConstr(sum(l[i] for i in range(390))<=4)
mod.addConstrs(z[i] == w[i]*l[i] for i in range(390))

# for i in range(6):
#     mod.addConstr(sum([y[i,k,j]*num_hours_or_daily_kj[k][j] for j in range(5) for k in range(5)])
#                   >= new_allocation_hr_t[i]-s[i])
#     mod.addConstr(s[i]>=0)
    
# for k in range(5):
#     for j in range(5):
#         mod.addConstr(sum([y[i,k,j] for i in range(6)]) == 1)
    
mod.setObjective(sum(z[i]*z[j]*C[i][j] for j in range(390) for i in range(390)), GRB.MINIMIZE)

mod.update()

mod.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 391 rows, 1170 columns and 780 nonzeros
Model fingerprint: 0xac912dd5
Model has 76245 quadratic objective terms
Model has 392 quadratic constraints
Variable types: 780 continuous, 390 integer (390 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e-06, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 8e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
  QRHS range       [5e-03, 1e+00]
Presolve removed 390 rows and 0 columns
Presolve time: 0.01s
Presolved: 1173 rows, 2340 columns, 3900 nonzeros
Presolved model has 780 SOS constraint(s)
Presolved model has 76245 quadratic objective terms
Variable types: 1560 continuous, 780 integer (780 binary)
Found heuristic solution: objec

In [52]:
z

{0: <gurobi.Var C780 (value 0.0)>,
 1: <gurobi.Var C781 (value 0.0)>,
 2: <gurobi.Var C782 (value 0.0)>,
 3: <gurobi.Var C783 (value 0.0)>,
 4: <gurobi.Var C784 (value 0.0)>,
 5: <gurobi.Var C785 (value 0.0)>,
 6: <gurobi.Var C786 (value 0.0)>,
 7: <gurobi.Var C787 (value 0.0)>,
 8: <gurobi.Var C788 (value 0.0)>,
 9: <gurobi.Var C789 (value 0.0)>,
 10: <gurobi.Var C790 (value 0.0)>,
 11: <gurobi.Var C791 (value 0.0)>,
 12: <gurobi.Var C792 (value 0.0)>,
 13: <gurobi.Var C793 (value 0.0)>,
 14: <gurobi.Var C794 (value 0.0)>,
 15: <gurobi.Var C795 (value 0.0)>,
 16: <gurobi.Var C796 (value 0.0)>,
 17: <gurobi.Var C797 (value 0.0)>,
 18: <gurobi.Var C798 (value 0.0)>,
 19: <gurobi.Var C799 (value 0.0)>,
 20: <gurobi.Var C800 (value 0.0)>,
 21: <gurobi.Var C801 (value 0.0)>,
 22: <gurobi.Var C802 (value 0.0)>,
 23: <gurobi.Var C803 (value 0.0)>,
 24: <gurobi.Var C804 (value 0.0)>,
 25: <gurobi.Var C805 (value 0.0)>,
 26: <gurobi.Var C806 (value 0.0)>,
 27: <gurobi.Var C807 (value 0.0)>,
 2

In [ ]:
118,285,348,389

In [59]:
for i in [118,285,348,389]:
    print(tickers[i],round(z[i].x*100,1),"%")
#     print()

CME 12.6 %
LLY 7.5 %
NVDA 4.4 %
BND 75.4 %
